In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import cenpy as cen

In [2]:
CENSUS_API_KEY = '6096dfc016847a1478466a805de4f84cd40ef87b'
cen.set_sitekey(CENSUS_API_KEY)

'/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/cenpy/SITEKEY.txt'

Some questions I want to explore from the ACS:

- Demographic changes on a county level in Georgia from 2013
    - Total amounts of people, and also by race
    - Median age and income by county

In [13]:
acs = cen.products.ACS(2013)
acs.tables

,description,columns
table_name,,
B00001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,[B00001_001E]
B00002,UNWEIGHTED SAMPLE HOUSING UNITS,[B00002_001E]
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B01003,TOTAL POPULATION,[B01003_001E]
...,...,...
C27007,MEDICAID/MEANS-TESTED PUBLIC COVERAGE BY SEX B...,"[C27007_001E, C27007_002E, C27007_003E, C27007..."
C27008,TRICARE/MILITARY HEALTH COVERAGE BY SEX BY AGE,"[C27008_001E, C27008_002E, C27008_003E, C27008..."
C27009,VA HEALTH CARE BY SEX BY AGE,"[C27009_001E, C27009_002E, C27009_003E, C27009..."


In [22]:
pd.set_option('display.max_colwidth', 1000)
acs.filter_variables('B02001')

,label,concept,predicateType,group,limit,predicateOnly,attributes,required,values
B02001_009E,Estimate!!Total!!Two or more races!!Two races including Some other race,RACE,int,B02001,0,NaN,"B02001_009EA,B02001_009M,B02001_009MA",NaN,NaN
B02001_007E,Estimate!!Total!!Some other race alone,RACE,int,B02001,0,NaN,"B02001_007EA,B02001_007M,B02001_007MA",NaN,NaN
B02001_008E,Estimate!!Total!!Two or more races,RACE,int,B02001,0,NaN,"B02001_008EA,B02001_008M,B02001_008MA",NaN,NaN
B02001_010E,"Estimate!!Total!!Two or more races!!Two races excluding Some other race, and three or more races",RACE,int,B02001,0,NaN,"B02001_010EA,B02001_010M,B02001_010MA",NaN,NaN
B02001_002E,Estimate!!Total!!White alone,RACE,int,B02001,0,NaN,"B02001_002EA,B02001_002M,B02001_002MA",NaN,NaN
B02001_001E,Estimate!!Total,RACE,int,B02001,0,NaN,"B02001_001EA,B02001_001M,B02001_001MA",NaN,NaN
B02001_006E,Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone,RACE,int,B02001,0,NaN,"B02001_006EA,B02001_006M,B02001_006MA",NaN,NaN
B02001_005E,Estimate!!Total!!Asian alone,RACE,int,B02001,0,NaN,"B02001_005EA,B02001_005M,B02001_005MA",NaN,NaN
B02001_004E,Estimate!!Total!!American Indian and Alaska Native alone,RACE,int,B02001,0,NaN,"B02001_004EA,B02001_004M,B02001_004MA",NaN,NaN
B02001_003E,Estimate!!Total!!Black or African American alone,RACE,int,B02001,0,NaN,"B02001_003EA,B02001_003M,B02001_003MA",NaN,NaN


In [25]:
vars_description = {
    'B01003_001E': 'pop_total',
    'B19013_001E': 'median_household_income',
    'B01002_001E': 'median_age',
    'B02001_002E': 'pop_white',
    'B02001_003E': 'pop_black_aa',
    'B02001_004E': 'pop_ai_an',
    'B02001_005E': 'pop_asian',
    'B02001_006E': 'pop_nh_pi'
}

df = acs.from_state(
    state = "Georgia",
    level = "county",
    return_geometry = True,
    variables = list(vars_description.keys())
)

/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of a

In [30]:
pd.set_option('display.max_colwidth', 200)
df.rename(columns = vars_description).head()

,GEOID,geometry,median_age,pop_total,pop_white,pop_black_aa,pop_ai_an,pop_asian,pop_nh_pi,median_household_income,NAME,state,county
0,13061,"POLYGON ((-9468390.610 3711479.410, -9468387.940 3711594.570, -9468420.440 3711684.750, -9468494.470 3711762.000, -9468537.660 3711795.590, -9468599.560 3711826.830, -9468626.160 3711840.430, -946...",40.7,3133.0,1104.0,1809.0,0.0,0.0,0.0,21192.0,"Clay County, Georgia",13,061
1,13307,"POLYGON ((-9422448.610 3775739.230, -9422448.500 3775747.900, -9422442.600 3776259.200, -9422440.480 3776449.210, -9422440.370 3776466.690, -9422440.040 3776490.210, -9422437.030 3776684.830, -942...",39.7,2769.0,1501.0,1258.0,0.0,0.0,0.0,35718.0,"Webster County, Georgia",13,307
2,13131,"POLYGON ((-9392716.290 3631140.560, -9392716.290 3631147.440, -9392715.510 3631188.090, -9392712.720 3631330.670, -9392711.950 3631378.320, -9392712.170 3631405.330, -9392713.060 3631486.230, -939...",38.4,25143.0,15955.0,7269.0,82.0,16.0,0.0,30571.0,"Grady County, Georgia",13,131
3,13193,"MULTIPOLYGON (((-9356817.530 3832029.430, -9356771.670 3832030.750, -9356783.910 3831743.340, -9356782.470 3831728.030, -9356780.910 3831714.960, -9356776.680 3831664.530, -9356785.580 3831429.930...",39.4,14455.0,5158.0,8778.0,40.0,220.0,0.0,30240.0,"Macon County, Georgia",13,193
4,13285,"POLYGON ((-9469340.500 3919018.500, -9469318.010 3919063.450, -9469280.500 3919133.010, -9469219.610 3919201.240, -9469139.680 3919244.740, -9469068.320 3919274.530, -9469007.650 3919297.810, -946...",36.1,67776.0,41377.0,22934.0,58.0,1117.0,6.0,41161.0,"Troup County, Georgia",13,285


In [33]:
years = range(2013, 2020)

vars_description = {
    'B01003_001E': 'pop_total',
    'B19013_001E': 'median_household_income',
    'B01002_001E': 'median_age',
    'B02001_002E': 'pop_white',
    'B02001_003E': 'pop_black_aa',
    'B02001_004E': 'pop_ai_an',
    'B02001_005E': 'pop_asian',
    'B02001_006E': 'pop_nh_pi'
}

df_list = []

for year in years:
    acs = cen.products.ACS(year)

    df = acs.from_state(
        state = "Georgia",
        level = "county",
        return_geometry = True,
        variables = list(vars_description.keys())
    ).to_crs({'init': 'epsg:4326'})

    df['year'] = year
    df.rename(columns = vars_description, inplace = True)
    df_list.append(df)

/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of a

KeyError: 'The requested Census Product shortcode (ACSDT5Y2020) was not found in the list of API shortcodes. Please check cenpy.explorer.available() to determine whether the API shortcode you have requested is correct.'

In [34]:
df_complete = pd.concat(df_list)

In [ ]:
df_complete.head()

In [36]:
df_complete.to_file('acs_georgia.geojson', driver = 'GeoJSON')

/home/ad/anaconda3/envs/comp-journal/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
